# Recommender Systems 2020/21

### Practice 4 - Cython for machine learning


### Cython is a superset of Python, allowing you to use C-like operations and import C code. Cython files (.pyx) are compiled and support static typing.

### Why do we use it (or any other compiled language)? If the code is written properly it is fast... I mean, FAST

In [1]:
import time
import numpy as np

### Let's implement something simple

In [2]:
def isPrime(n):
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [3]:
print("Is prime 2? {}".format(isPrime(2)))
print("Is prime 3? {}".format(isPrime(3)))
print("Is prime 5? {}".format(isPrime(5)))
print("Is prime 15? {}".format(isPrime(15)))
print("Is prime 20? {}".format(isPrime(20)))

Is prime 2? True
Is prime 3? True
Is prime 5? True
Is prime 15? False
Is prime 20? False


In [4]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 8.25 sec


#### Load Cython magic command, this takes care of the compilation step. If you are writing code outside Jupyter you'll have to compile using other tools. See at the end of the notebook for details.

In [5]:
%load_ext Cython

#### Declare Cython function, paste the same code as before. The function will be compiled and then executed with a Python interface

In [6]:
%%cython
def isPrime(n):
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [7]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 3.77 sec


#### As you can see by just compiling the same code we got some improvement.
#### To go seriously higher, we have to use some static tiping

In [8]:
%%cython
# Declare the tipe of the arguments
def isPrime(long n):
    
    # Declare index of for loop
    cdef long i
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [9]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 0.27 sec


#### Cython code with two tipe declaration, for n and i, runs 50x faster than Python

#### Main benefits of Cython:
* Compiled, no interpreter
* Static typing, no overhead
* Fast loops, no need to vectorize. Vectorization sometimes performes lots of useless operations
* Numpy, which is fast in python, when opertions are not vectorizable often becomes slooooow compared to a carefully written Cython code

## SLIM MSE with Cython

#### Load the usual data.

In [10]:
from Notebooks_utils.data_splitter import train_test_holdout
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10217, feature occurrences: 108563, density 9.95E-04
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10237, feature occurrences: 130127, density 1.19E-03




In [11]:
URM_train

<69878x10681 sparse matrix of type '<class 'numpy.float64'>'
	with 8002091 stored elements in Compressed Sparse Row format>

### What do we need for a SLIM MSE?

* Item-Item similarity matrix
* Computing prediction
* Update rule
* Training loop and some patience


In [12]:
n_users, n_items = URM_train.shape

## Step 1: We create a dense similarity matrix, initialized as zero

In [13]:
item_item_S = np.zeros((n_items, n_items), dtype = np.float)
item_item_S

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Step 2: We sample an interaction and compute the prediction of the current SLIM model

In [14]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

7356431

In [15]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(64496, 16, 3.0)

In [16]:
predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
predicted_rating

0.0

#### The first predicted rating is zero, of course, the model is "empty"

### Step 3: We compute the prediction error and update the item-item similarity

In [17]:
prediction_error = rating - predicted_rating
prediction_error

3.0

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values in the item-item similarity matrix

### Which item similarities we modify? Only those we used to compute the prediction, i.e., only the items in the profile of the sampled user. 

In [18]:
items_in_user_profile = URM_train[user_id].indices
items_in_user_profile

array([   3,    4,    5,    7,   11,   14,   16,   17,   19,   20,   25,
         26,   29,   30,   31,   32,   35,   36,   37,   38,   39,   40,
         50,   73,   75,   76,   77,   78,   80,   84,   85,   90,   91,
         93,   96,  102,  113,  115,  126,  133,  137,  140,  147,  150,
        156,  160,  175,  177,  178,  179,  183,  185,  188,  194,  195,
        196,  224,  227,  228,  231,  235,  241,  248,  253,  254,  255,
        256,  258,  261,  300,  314,  317,  318,  320,  322,  324,  337,
        340,  345,  347,  369,  371,  383,  388,  398,  399,  400,  401,
        403,  406,  410,  411,  413,  414,  415,  417,  418,  423,  424,
        425,  427,  428,  437,  447,  450,  454,  457,  463,  470,  474,
        488,  500,  503,  506,  507,  513,  516,  517,  519,  520,  523,
        528,  533,  540,  561,  566,  577,  595,  596,  599,  611,  612,
        615,  617,  621,  622,  623,  627,  629,  633,  992, 1021, 1024,
       1025, 1033, 1041, 1057, 1062, 1066, 1073, 10

#### Apply the update rule

In [19]:
item_item_S[items_in_user_profile,item_id]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [20]:
learning_rate = 1e-4

item_item_S[items_in_user_profile,item_id] += prediction_error * learning_rate

In [21]:
item_item_S[items_in_user_profile,item_id]

array([0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
       0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003,
      

### Let's check what the new prediction for the same user-item interaction would be

In [22]:
predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
predicted_rating

0.2601000000000008

### The value is not zero anymore, but higher, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### Let's put all together in a training loop.

In [23]:
URM_train_coo = URM_train.tocoo()
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

learning_rate = 1e-6

loss = 0.0
start_time = time.time()
for sample_num in range(100000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2
    
    # Update model, in this case the similarity
    items_in_user_profile = URM_train[user_id].indices
    item_item_S[items_in_user_profile,item_id] += prediction_error * learning_rate
    
    # Print some stats
    if (sample_num +1)% 5000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 5000 in 2.20 seconds, loss is 13.45. Samples per second 2277.32
Iteration 10000 in 4.27 seconds, loss is 13.52. Samples per second 2340.46
Iteration 15000 in 6.34 seconds, loss is 13.48. Samples per second 2366.01
Iteration 20000 in 8.42 seconds, loss is 13.46. Samples per second 2373.91
Iteration 25000 in 10.47 seconds, loss is 13.46. Samples per second 2386.63
Iteration 30000 in 12.48 seconds, loss is 13.46. Samples per second 2404.40
Iteration 35000 in 14.47 seconds, loss is 13.43. Samples per second 2419.43
Iteration 40000 in 16.39 seconds, loss is 13.43. Samples per second 2440.76
Iteration 45000 in 18.34 seconds, loss is 13.44. Samples per second 2453.73
Iteration 50000 in 20.29 seconds, loss is 13.41. Samples per second 2464.08
Iteration 55000 in 22.24 seconds, loss is 13.39. Samples per second 2473.06
Iteration 60000 in 24.21 seconds, loss is 13.39. Samples per second 2478.54
Iteration 65000 in 26.17 seconds, loss is 13.38. Samples per second 2483.77
Iteration 70000 i

### What do we see? The loss oscillates over time, sometimes it goes down sometimes up.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [24]:
def train_one_epoch(URM_train, item_item_S, learning_rate):
    
    URM_train_coo = URM_train.tocoo()

    loss = 0.0
    start_time = time.time()
    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        items_in_user_profile = URM_train[user_id].indices
        item_item_S[items_in_user_profile,item_id] += prediction_error * learning_rate

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))
         
            # Stop training because this implementation is too slow
            print("\tStopping the epoch early because this implementation is too slow")
            return item_item_S
            
    return item_item_S

In [25]:
n_items = URM_train.shape[1]
learning_rate = 1e-6
    
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

for n_epoch in range(5):
    item_item_S = train_one_epoch(URM_train, item_item_S, learning_rate)
    

Iteration 5000 in 2.18 seconds, loss is 13.51. Samples per second 2292.99
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 2.01 seconds, loss is 13.40. Samples per second 2493.12
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 1.99 seconds, loss is 13.54. Samples per second 2514.44
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 2.00 seconds, loss is 13.44. Samples per second 2495.61
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 2.00 seconds, loss is 13.47. Samples per second 2501.86
	Stopping the epoch early because this implementation is too slow


In [26]:
item_item_S

array([[1.64932891e-05, 1.49683995e-05, 2.59444045e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.34968958e-05, 1.11239779e-04, 4.98784905e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.99529948e-06, 5.83871111e-05, 1.26274733e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### How do we use this similarity? As in a simple item-based KNN

### Let's estimate the training time. Say we train for 10 epochs and we have 8M interactions in the train data...

In [27]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 31890.14 seconds, or 531.50 minutes


### ... ehm, 10 hours
### Unacceptable!

### Let's rewrite the loop with some smarter use of the data structures. In particular:
* Use the indptr/indices data structures to get the seen items
* Not much else we can do with this tools

In [28]:
URM_train_coo = URM_train.tocoo()
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

learning_rate = 1e-6

loss = 0.0
start_time = time.time()
for sample_num in range(100000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2
    
    # Update model, in this case the similarity
    items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
    item_item_S[items_in_user_profile,item_id] += prediction_error * learning_rate
    
    # Print some stats
    if (sample_num +1)% 5000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 5000 in 1.79 seconds, loss is 13.38. Samples per second 2795.70
Iteration 10000 in 3.45 seconds, loss is 13.36. Samples per second 2896.42
Iteration 15000 in 5.08 seconds, loss is 13.33. Samples per second 2954.13
Iteration 20000 in 6.73 seconds, loss is 13.34. Samples per second 2973.66
Iteration 25000 in 8.34 seconds, loss is 13.34. Samples per second 2996.23
Iteration 30000 in 9.96 seconds, loss is 13.36. Samples per second 3013.29
Iteration 35000 in 11.58 seconds, loss is 13.35. Samples per second 3023.24
Iteration 40000 in 13.19 seconds, loss is 13.36. Samples per second 3033.04
Iteration 45000 in 14.81 seconds, loss is 13.34. Samples per second 3037.84
Iteration 50000 in 16.43 seconds, loss is 13.34. Samples per second 3042.42
Iteration 55000 in 18.06 seconds, loss is 13.32. Samples per second 3044.67
Iteration 60000 in 19.67 seconds, loss is 13.32. Samples per second 3049.80
Iteration 65000 in 21.29 seconds, loss is 13.31. Samples per second 3053.43
Iteration 70000 in 

In [29]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 26107.35 seconds, or 435.12 minutes


### We now got 7 hours, just as bad as before

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [30]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):
    
    URM_train_coo = URM_train.tocoo()
    n_items = URM_train.shape[1]
    
    item_item_S = np.zeros((n_items, n_items), dtype = np.float16)

    learning_rate = 1e-6

    loss = 0.0
    start_time = time.time()
    for sample_num in range(100000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        item_item_S[items_in_user_profile,item_id] += prediction_error * learning_rate

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))   
    
    return loss, samples_per_second

In [31]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 5000 in 1.44 seconds, loss is 13.34. Samples per second 3461.71
Iteration 10000 in 2.88 seconds, loss is 13.39. Samples per second 3475.30
Iteration 15000 in 4.34 seconds, loss is 13.40. Samples per second 3455.00
Iteration 20000 in 5.82 seconds, loss is 13.44. Samples per second 3436.06
Iteration 25000 in 7.31 seconds, loss is 13.46. Samples per second 3418.70
Iteration 30000 in 8.82 seconds, loss is 13.45. Samples per second 3402.60
Iteration 35000 in 10.33 seconds, loss is 13.45. Samples per second 3387.57
Iteration 40000 in 11.85 seconds, loss is 13.44. Samples per second 3374.40
Iteration 45000 in 13.39 seconds, loss is 13.40. Samples per second 3361.20
Iteration 50000 in 14.93 seconds, loss is 13.42. Samples per second 3348.03
Iteration 55000 in 16.48 seconds, loss is 13.41. Samples per second 3336.32
Iteration 60000 in 18.07 seconds, loss is 13.39. Samples per second 3319.81
Iteration 65000 in 19.67 seconds, loss is 13.39. Samples per second 3304.45
Iteration 70000 in 

In [32]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 24594.45 seconds, or 409.91 minutes


### Still far too time consuming
### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix


### We now use types for all main variables


In [33]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_items = URM_train.shape[1]

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = 1e-6
    cdef double loss = 0.0
    cdef long start_time = time.time()
    cdef double predicted_rating, prediction_error
    cdef int[:] items_in_user_profile
    cdef int index, sample_num

    for sample_num in range(100000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        predicted_rating = 0.0

        for index in items_in_user_profile:
            predicted_rating += item_item_S[index,item_id]

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        for index in items_in_user_profile:
            item_item_S[index,item_id] += prediction_error * learning_rate

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [34]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 5000 in 1.97 seconds, loss is 13.42. Samples per second 2543.59
Iteration 10000 in 2.98 seconds, loss is 13.48. Samples per second 3352.68
Iteration 15000 in 3.98 seconds, loss is 13.45. Samples per second 3764.39
Iteration 20000 in 4.96 seconds, loss is 13.47. Samples per second 4029.21
Iteration 25000 in 5.99 seconds, loss is 13.49. Samples per second 4173.76
Iteration 30000 in 7.02 seconds, loss is 13.51. Samples per second 4271.77
Iteration 35000 in 8.04 seconds, loss is 13.49. Samples per second 4354.91
Iteration 40000 in 9.06 seconds, loss is 13.46. Samples per second 4415.03
Iteration 45000 in 10.08 seconds, loss is 13.47. Samples per second 4464.28
Iteration 50000 in 11.07 seconds, loss is 13.46. Samples per second 4514.64
Iteration 55000 in 12.09 seconds, loss is 13.46. Samples per second 4550.67
Iteration 60000 in 13.07 seconds, loss is 13.46. Samples per second 4589.20
Iteration 65000 in 14.09 seconds, loss is 13.46. Samples per second 4614.43
Iteration 70000 in 15

In [35]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 16857.31 seconds, or 280.96 minutes


### This is why we use it for machine learning algorithms...

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
####  For example, this operation: user_id = URM_train_coo.row[sample_index]
#### Becomes:
#### cdef int user_id
#### cdef int[:] URM_train_coo_row = URM_train_coo.row
#### user_id = URM_train_coo_row[sample_index]

### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()

## To obtain a more reliable speed estimate we increase the number of samples and the print step by a factor of 10

In [36]:
%%cython

import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    cdef int n_items = URM_train.shape[1]
    cdef int n_interactions = URM_train.nnz
    cdef int[:] URM_train_row = URM_train_coo.row
    cdef int[:] URM_train_col = URM_train_coo.col
    cdef double[:] URM_train_data = URM_train_coo.data
    cdef int[:] URM_train_indices = URM_train.indices
    cdef int[:] URM_train_indptr = URM_train.indptr

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = 1e-6
    cdef double loss = 0.0
    cdef long start_time = time.time()
    cdef double rating, predicted_rating, prediction_error
    cdef int start_profile, end_profile
    cdef int index, sample_num, user_id, item_id, seen_item_id

    for sample_num in range(1000000):

        # Randomly pick sample
        index = rand() % n_interactions

        user_id = URM_train_row[index]
        item_id = URM_train_col[index]
        rating = URM_train_data[index]

        # Compute prediction
        start_profile = URM_train_indptr[user_id]
        end_profile = URM_train_indptr[user_id+1]
        predicted_rating = 0.0

        for index in range(start_profile, end_profile):
            seen_item_id = URM_train_indices[index]
            predicted_rating += item_item_S[seen_item_id,item_id]

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        for index in range(start_profile, end_profile):
            seen_item_id = URM_train_indices[index]
            item_item_S[seen_item_id,item_id] += prediction_error * learning_rate

        # Print some stats
        if (sample_num +1)% 50000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [37]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 50000 in 0.80 seconds, loss is 13.91. Samples per second 62854.95
Iteration 100000 in 1.26 seconds, loss is 13.88. Samples per second 79270.34
Iteration 150000 in 1.73 seconds, loss is 13.84. Samples per second 86578.50
Iteration 200000 in 2.20 seconds, loss is 13.85. Samples per second 90803.50
Iteration 250000 in 2.68 seconds, loss is 13.82. Samples per second 93437.51
Iteration 300000 in 3.15 seconds, loss is 13.80. Samples per second 95310.36
Iteration 350000 in 3.62 seconds, loss is 13.78. Samples per second 96774.95
Iteration 400000 in 4.08 seconds, loss is 13.75. Samples per second 98023.24
Iteration 450000 in 4.55 seconds, loss is 13.73. Samples per second 98864.34
Iteration 500000 in 5.02 seconds, loss is 13.70. Samples per second 99607.19
Iteration 550000 in 5.49 seconds, loss is 13.67. Samples per second 100241.58
Iteration 600000 in 5.96 seconds, loss is 13.64. Samples per second 100725.70
Iteration 650000 in 6.42 seconds, loss is 13.61. Samples per second 101201.

In [38]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 776.56 seconds, or 12.94 minutes


### As the source code gets less readable due to the addition of types and native C functions, it also gets remarkably faster

### We started from a naive python implementation which took 10 hours (2k samples per second) and we now have a Cython one with takes 12 minutes (100k samples per second).

In [39]:
%%cython

import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, learning_rate_input, n_epochs):

    URM_train_coo = URM_train.tocoo()
    cdef int n_items = URM_train.shape[1]
    cdef int n_interactions = URM_train.nnz
    cdef int[:] URM_train_row = URM_train_coo.row
    cdef int[:] URM_train_col = URM_train_coo.col
    cdef double[:] URM_train_data = URM_train_coo.data
    cdef int[:] URM_train_indices = URM_train.indices
    cdef int[:] URM_train_indptr = URM_train.indptr

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = learning_rate_input
    cdef double loss = 0.0
    cdef long start_time
    cdef double rating, predicted_rating, prediction_error
    cdef int start_profile, end_profile
    cdef int index, sample_num, user_id, item_id, seen_item_id
    
    for n_epoch in range(n_epochs):
        
        loss = 0.0
        start_time = time.time()
        
        for sample_num in range(n_interactions):

            # Randomly pick sample
            index = rand() % n_interactions

            user_id = URM_train_row[index]
            item_id = URM_train_col[index]
            rating = URM_train_data[index]

            # Compute prediction
            start_profile = URM_train_indptr[user_id]
            end_profile = URM_train_indptr[user_id+1]
            predicted_rating = 0.0

            for index in range(start_profile, end_profile):
                seen_item_id = URM_train_indices[index]
                predicted_rating += item_item_S[seen_item_id,item_id]

            # Compute prediction error, or gradient
            prediction_error = rating - predicted_rating
            loss += prediction_error**2

            # Update model, in this case the similarity
            for index in range(start_profile, end_profile):
                seen_item_id = URM_train_indices[index]
                item_item_S[seen_item_id,item_id] += prediction_error * learning_rate

#             # Print some stats
#             if (sample_num +1)% 1000000 == 0:
#                 elapsed_time = time.time() - start_time
#                 samples_per_second = sample_num/elapsed_time
#                 print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

            
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
     
        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/sample_num, samples_per_second))

    return np.array(item_item_S), loss/sample_num, samples_per_second

In [40]:
n_items = URM_train.shape[1]
learning_rate = 1e-3
    
item_item_S, loss, samples_per_second = train_multiple_epochs(URM_train, learning_rate, 10)

Epoch 1 complete in in 77.13 seconds, loss is 1.392E-01. Samples per second 103751.66
Epoch 2 complete in in 76.25 seconds, loss is 2.346E-04. Samples per second 104940.35
Epoch 3 complete in in 76.57 seconds, loss is 3.435E-05. Samples per second 104505.40
Epoch 4 complete in in 76.99 seconds, loss is 6.974E-06. Samples per second 103933.45
Epoch 5 complete in in 77.39 seconds, loss is 1.601E-06. Samples per second 103397.12
Epoch 6 complete in in 77.01 seconds, loss is 3.957E-07. Samples per second 103906.80
Epoch 7 complete in in 77.28 seconds, loss is 1.013E-07. Samples per second 103545.90
Epoch 8 complete in in 72.25 seconds, loss is 2.661E-08. Samples per second 110749.86
Epoch 9 complete in in 71.49 seconds, loss is 7.077E-09. Samples per second 111928.61
Epoch 10 complete in in 71.47 seconds, loss is 1.882E-09. Samples per second 111959.84


In [41]:
item_item_S

array([[0.39046878, 0.24915104, 0.33083516, ..., 0.        , 0.        ,
        0.        ],
       [0.24919639, 0.70542567, 0.25620503, ..., 0.        , 0.        ,
        0.        ],
       [0.24947692, 0.28547543, 0.71379698, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Note how the loss decreased?

## How to use Cython outside a notebook

### Step1: Create a .pyx file and write your code

### Step2: Create a compilation script "compileCython.py" with the following content

In [ ]:
# This code will not run in a notebook cell

try:
    from setuptools import setup
    from setuptools import Extension
except ImportError:
    from distutils.core import setup
    from distutils.extension import Extension


from Cython.Distutils import build_ext
import numpy
import sys
import re


if len(sys.argv) != 4:
    raise ValueError("Wrong number of paramethers received. Expected 4, got {}".format(sys.argv))


# Get the name of the file to compile
fileToCompile = sys.argv[1]

# Remove the argument from sys argv in order for it to contain only what setup needs
del sys.argv[1]

extensionName = re.sub("\.pyx", "", fileToCompile)


ext_modules = Extension(extensionName,
                [fileToCompile],
                extra_compile_args=['-O3'],
                include_dirs=[numpy.get_include(),],
                )

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[ext_modules]
)


### Step3: Compile your code with the following command 

python compileCython.py Cython_examples\SLIM_MSE_fastest.pyx build_ext --inplace

### Step4: Generate cython report and look for "yellow lines". The report is an .html file which represents how many operations are necessary to translate each python operation in cython code. If a line is white, it has a direct C translation. If it is yellow it will require many indirect steps that will slow down execution. Some of those steps may be inevitable, some may be removed via static typing.

### IMPORTANT: white does not mean fast!! If a system call is involved that part might be slow anyway.

cython -a Cython_examples\SLIM_MSE_fastest.pyx

### Step5: Add static types and C functions to remove "yellow" lines.

#### If you use a variable only as a C object, use primitive tipes 
cdef int namevar

def double namevar

cdef float namevar

#### If you call a function only within C code, use a specific declaration "cdef"

cdef function_name(self, int param1, double param2):
...



## Step6: Iterate step 4 and 5 until you are satisfied with how clean your code is, then compile. An example of non optimized code can be found in the source folder of this notebook with the _SLOW suffix

## Step7: the compilation generates a file wose name is something like "SLIM_MSE_fastest.cp36-win_amd64.pyd" and tells you the source file, the architecture it is compiled for and the OS

## Step8: Import and use the compiled file as if it were any python object, function or class

In [ ]:
from Cython_examples.SLIM_MSE_fastest import train_multiple_epochs

train_multiple_epochs(URM_train, 1e-3, 1)

# A few warnings on ML algorithms

### - Why do we bother with KNNs if we have ML?
#### Because sometimes ML algorithms work better than heuristic ones, sometimes they do not

### - ML algorithms are always best because they learn from the data
#### Not really... Yes they learn from the data but the data is sometimes too noisy, too sparse and does not yeld good results. There is plenty of examples of cases where ML algorithms are not the best choice.

### - We should use this complex model because it can in theory approximate any function!!
#### Theory is important but... does it work in practice? Often complex modes are veeeery difficult to train and you need to use lots of tricks: adaptive gradients, data normalization, careful initialization and crafted batches...

### - I have trained my model for 2 epochs but the result is not great
#### Have you just used the default learning rate or have you optimized it? Why did you stop after 2 epochs? You may need 100s of epochs

### - If I select a high learning rate (maybe 1e-3) after 5 epochs I get a result wich is not very good, if I use a lower learning rate (maybe 1e-6) the result is much worse
#### Of course, the lower the learning rate the slower the training process but the best solution you may find. Again, you may need 100s of epochs

### - Training and optimizing the hyperparameters of this ML model takes several hours, what am I doing wrong?
#### Probably nothing, ML is computationally expensive and takes time... A few hours is a normal timespan. Sometimes the end result is still not satisfactory.